# Travail de mise en forme des données
Données téléchargées sur : http://fisher.stats.uwo.ca/faculty/aim/epubs/mhsets/thompsto/

Nous avons les données qui sont disposées par quart de mois, soit 48 données par année, sur 30 ans (1953 à 1982), donc un total de 1440 observations


In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
from torch import optim
from attrdict import AttrDict
import tqdm

device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print(device)

In [ ]:
# Définition des constantes
N_UNITES_PAR_MOIS = 4
N_UNITES_PAR_AN = 12 * N_UNITES_PAR_MOIS
N_ANNEE_DATASET = 30
N_UNITES_DANS_DATASET = N_ANNEE_DATASET * N_UNITES_PAR_AN

#On crée la séquence de temps discrétiser sur 30 ans et chaque année elle même discrétiser sur 48 quarts de mois
Big_Data = np.empty(shape=(N_UNITES_DANS_DATASET,5))
for i in range(N_ANNEE_DATASET):
    for j in range(N_UNITES_PAR_AN):
        Big_Data[j+i*N_UNITES_PAR_AN][0]=1953+i
        Big_Data[j+i*N_UNITES_PAR_AN][1]=j+1

#On importe les valeurs des fichiers, et on traite les donnéees de telle sorte à n'avoir qu'une colonne de 1440 observations
Files = ["lacstjin.txt","lacstjra.txt","lacstjsn.txt"]
for count,file in enumerate(Files) :
    data = np.reshape(np.loadtxt("./Data/"+file), newshape=(1,1440))
    Big_Data[:,count+2] = (data.copy() - data.min(axis=1))/(data.max(axis=1)-data.min(axis=1)) 
    
Big_Data = pd.DataFrame(Big_Data, columns=['Years','Quarters','Inflow','Rainfall','Melt of Ice'])

# Premiers Graphiques sur les données brutes

Ici on chercher à sortir des graphiques sur les données en mettant le temps en abcisse afin d'apporter des premières réflexions

## Tracé de l'ensemble du jeu de donnée

In [ ]:
# Création de la liste des années à accoller aux quarts de mois
Time_Stamp = ['Y:'+str(1953+i) for i in range(N_ANNEE_DATASET)]
Time_Stamp_x = [(i+.5)*N_UNITES_PAR_AN for i in range(N_ANNEE_DATASET)]
timing = [i for i in range(N_UNITES_DANS_DATASET)]

titles  = ['Inflow','Rainfall','Melt of Ice']
colors = ['blue','green','orange']

#Plot des données
for t,c in zip(titles,colors):
    fig, ax = plt.subplots(figsize=(40,5))
    ax.plot(timing, Big_Data[t], color = c)
    ax.set_xlim(0,N_UNITES_DANS_DATASET)
    ax.set_ylim(0,1)
    for i in range(N_ANNEE_DATASET):
        if i%2==0:
            ax.axvspan(i*N_UNITES_PAR_AN, (i+1)*N_UNITES_PAR_AN, ec='black',fill=False)
    
    plt.xticks(Time_Stamp_x,Time_Stamp)
    plt.margins(0.2)
    plt.subplots_adjust(bottom=0.15)
    plt.title(f'Quarters Monthly {t} over the 30 years')
    plt.show()

## Comparaison par année

On remarque dans la mise en graphique des données brutes une certaine périodicité des valeurs, dans les prochaines étapes nous allons superposer les données par année afin de vérifier ce propos :

In [ ]:
# Liste des abscisses fixe sur l'année
X = Big_Data['Quarters'].iloc[0:N_UNITES_PAR_AN]

for title in titles:
    #Plot de Inflow sur fenêtre d'un an
    fig, ax = plt.subplots(figsize=(20,10))
    for i in range(N_ANNEE_DATASET):
        Y = Big_Data[title].iloc[N_UNITES_PAR_AN*i:N_UNITES_PAR_AN*(i+1)]
        labels = Big_Data['Years'].iloc[N_UNITES_PAR_AN*i]
        ax.plot(X , Y, label=labels)
    #Moyenne transversale
    title_mean_serie = [np.mean([Big_Data[title].iloc[i*N_UNITES_PAR_AN+j] for i in range(N_ANNEE_DATASET)]) for j in range(N_UNITES_PAR_AN)]
    ax.plot(X, title_mean_serie, color='black', linestyle='dashed', linewidth=3,label="Mean")
    plt.legend()
    plt.title(f'Quarters monthly {title} per year')

## Superposition des données 

Maintenant que l'on a montré que les données suivent la réalité scientifique, avec des tendances saisonnière remarquable sur la plupart des années, nous allons superposer les données normalisées de quelques années afin d'observer les liens de causes à effets sur le débit du fleuve

In [ ]:
#Choix des années à plot
I = [0,9,19,29]

#Plot des 4 années selectionnées
fig, ax = plt.subplots(figsize=(20,5))

# Liste des abscisses fixe sur l'année
X = Big_Data['Quarters'].iloc[0:N_UNITES_PAR_AN]

for l,i in enumerate(I):
    plt.subplot(1,len(I),l+1)
    for title,color in zip(titles,colors):
        plt.plot(X, Big_Data[title].iloc[N_UNITES_PAR_AN*i:N_UNITES_PAR_AN*(i+1)], label=title,color=color, linewidth=1)
    plt.legend()
    plt.title('Year %s' % (int(Big_Data['Years'].iloc[N_UNITES_PAR_AN*i])))

# Préparation des données
Découpage des données d'observations et d'actions pour les phases:
- d'entraînement sur les 26 premières années
- de validation sur la 27 et 28 ème année
- de test sur les 2 années restantes

Sur chacun de ces découpages, nous allons faire glisser une fenêtre correspondant à certain nombre de mois afin d'augmenter nos données.

In [ ]:
def dataset_train_valid_test (X,U,config):
    
    # Données de configuration extraites afin qu'elles soient explicites
    N_ANNEE_ENTRAINEMENT = config.n_annee_entrainement*N_UNITES_PAR_AN
    N_ANNEE_VALIDATION = N_ANNEE_ENTRAINEMENT + config.n_annee_validation*N_UNITES_PAR_AN
    batch_size = config.n_mois*N_UNITES_PAR_MOIS
    
    # Découpage des données pour le jeu d'entrainement
    X_train = np.lib.stride_tricks.sliding_window_view(X[:N_ANNEE_ENTRAINEMENT],batch_size)
    U_train = np.lib.stride_tricks.sliding_window_view(U[:N_ANNEE_ENTRAINEMENT],(batch_size,U.shape[1]))
    U_train = U_train.reshape(N_ANNEE_ENTRAINEMENT-batch_size+1,batch_size,U.shape[1])[:-1]
    
    # Découpage des données pour le jeu de validation
    X_validation = np.lib.stride_tricks.sliding_window_view(X[N_ANNEE_ENTRAINEMENT-batch_size:N_ANNEE_VALIDATION],batch_size)
    U_validation = np.lib.stride_tricks.sliding_window_view(U[N_ANNEE_ENTRAINEMENT-batch_size:N_ANNEE_VALIDATION],(batch_size,U.shape[1]))
    U_validation = U_validation.reshape(N_ANNEE_VALIDATION-N_ANNEE_ENTRAINEMENT+1,batch_size,U.shape[1])[:-1]
    
    # Découpage des données pour le jeu de test
    X_test = np.lib.stride_tricks.sliding_window_view(X[N_ANNEE_VALIDATION-batch_size:],batch_size)
    U_test = np.lib.stride_tricks.sliding_window_view(U[N_ANNEE_VALIDATION-batch_size:],(batch_size,U.shape[1]))
    U_test = U_test.reshape(N_UNITES_DANS_DATASET-N_ANNEE_VALIDATION+1,batch_size,U.shape[1])[:-1]
    
    return X_train, U_train, X_validation, U_validation, X_test, U_test

In [ ]:
X = Big_Data['Inflow'].to_numpy()
U = Big_Data[['Inflow','Rainfall','Melt of Ice']].to_numpy()

config_data = AttrDict({
    "n_mois": 3,
    "n_annee_entrainement": 26,
    "n_annee_validation": 2
    })

X_train, U_train, X_validation, U_validation, X_test, U_test = dataset_train_valid_test (X,U,config_data)

X_train_torch = torch.Tensor(X_train).unsqueeze(-1).to(device)
U_train_torch = torch.Tensor(U_train).to(device)
X_valid_torch = torch.Tensor(X_validation).unsqueeze(-1).to(device)
U_valid_torch = torch.Tensor(U_validation).to(device)

# Deep Kalman Filter

Mise en place d'un réseau d'inférence du filtre Kalman, avec une paramétrisation par MLP

In [ ]:
class Decodeur (nn.Module):
    """
    Retourne des observations x_t selon une loi de distribution (à définir soi-même, ici gaussien) avec des paramètres qui sont fonctions de l'espace latent z_t
    """
    def __init__(self,z_dim,dim_cc,decodeur_dim_sortie): 
        """
        z_dim correspond à la dimension de l'espace latent z_t
        dim_cc correspond à la dimension des couches cachées complétements connectées
        decodeur_dim_sortie correspond aux nombres de sorties souhaitées
        """
        super(Decodeur, self).__init__()
        self.z_to_cc = nn.Linear(z_dim, dim_cc)
        self.cc_to_cc = nn.Linear(dim_cc, dim_cc)
        self.cc_to_mu = nn.Linear(dim_cc, decodeur_dim_sortie)
        
        self.relu = nn.ReLU()
    
    def forward(self, z_t):
        """
        On retourne uniquement le vecteur des moyennes paramètres des gaussiennes
        """
        cc1 = self.relu(self.z_to_cc(z_t))
        cc2 = self.relu(self.cc_to_cc(cc1))
        mu = self.cc_to_mu(cc2)

        return mu

class Transition(nn.Module):
    """
    Distribution de l'espace latent z_{t-1} à z_t
    """
    def __init__(self, z_dim, dim_cc):
        """
        z_dim correspond à la dimension de l'espace latent z_t
        dim_cc correspond à la dimension des couches cachées complétements connectées
        """
        super(Transition, self).__init__()
        self.z_to_cc = nn.Linear(z_dim, dim_cc)
        self.cc_to_cc = nn.Linear(dim_cc, dim_cc)
        self.cc_to_mu = nn.Linear(dim_cc, z_dim)
        self.cc_to_sig = nn.Linear(dim_cc, z_dim)

        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()

    def forward(self, z_t_1):
        """
        On retourne la structure de l'espace latent
        """

        cc1 = self.relu(self.z_to_cc(z_t_1))
        cc2 = self.relu(self.cc_to_cc(cc1))

        mu = self.cc_to_mu(cc2)
        sigma = self.softplus(self.cc_to_sig(cc2))

        return mu, sigma
    
class Posterieur(nn.Module):
    """
    Ajustement des paramètres de distribution sortie de Transition avec l'intégration des variables d'action, c'est ici que l'on met à profit le principe des filtres de Kalman
    """
    def __init__(self, z_dim, cc_dim, actions_dim):
        """
        z_dim correspond à la dimension de l'espace latent z_t
        dim_cc correspond à la dimension des couches cachées complétements connectées
        actions_dim correspond à la dimension des actions prises en comptes dans le postérieur
        """
        super(Posterieur, self).__init__()
        self.z_plus_action_to_cc = nn.Linear(2*z_dim + actions_dim, cc_dim)
        self.cc_to_cc = nn.Linear(cc_dim, cc_dim)
        self.cc_to_mu = nn.Linear(cc_dim, z_dim)
        self.cc_to_sig = nn.Linear(cc_dim, z_dim)
        self.relu = nn.ReLU()
        self.softplus = nn.Softplus()

    def forward(self, z_mu, z_sig, action_t):
        """
        On retourne les nouveaux paramètres de la distribution réajustés
        """
        cc1 = self.relu(self.z_plus_action_to_cc(torch.cat((z_mu, z_sig, action_t), dim=-1)))
        cc2 = self.relu(self.cc_to_cc(cc1))

        mu = self.cc_to_mu(cc2)
        sig = self.softplus(self.cc_to_sig(cc2))

        return mu, sig

In [ ]:
class DeepKalmanFilter(nn.Module):
    """
    Mise en place du Deep Kalman Filter
    """
    def __init__(self, config):
        """
        Variables nécessaires à l'initialisation des modules
        z_dim                   : dimension de l'espace latent z_t
        decodeur_dim_cachee     : dimension des couches cachées du décodeur entre l'espace latent et la sortie
        decodeur_dim_sortie    : dimension de la sortie du décodeur
        transition_dim_cachee   : dimension des couches cachées du réseau de transition entre l'espace latent z_{t-1} et le postérieur
        actions_dim             : dimension des actions prises en compte dans le réseau de postérieur
        posterieur_dim_cachee   : dimension des couches cachées du réseau de postérieur entre la (transition + actions) et l'espace latent z_t
        
        """
        super(DeepKalmanFilter, self).__init__()

        self.decodeur = Decodeur(
            config.z_dim, 
            config.decodeur_dim_cachee, 
            config.decodeur_dim_sortie
            )
        
        self.transition = Transition(
            config.z_dim, 
            config.transition_dim_cachee
            )

        self.posterieur = Posterieur(
            config.z_dim,
            config.posterieur_dim_cachee,
            config.actions_dim
        )
        # Initialisation du premier espace latent
        self.z_q_0 = nn.Parameter(torch.zeros(config.z_dim))
        # Initialisation de l'écart type qui servira à la reconstruction des x_t via le décodeur
        self.decodeur_sigma = nn.Parameter(config.decodeur_constante*torch.ones(config.decodeur_dim_sortie))
        #self.decodeur_sigma = nn.Parameter(torch.ones(config.decodeur_dim_sortie))
        # Dictionnaire contenant toutes les informations nécessaires au fonctionnement de la classe
        self.config = config

    @staticmethod
    def reparametrization(mu, sig):
        """
        Astuce de reparamétrisation après la sortie du postérieur pour permettre la rétropropagation du réseau dans son ensemble
        """
        return mu + torch.randn_like(sig) * sig

    @staticmethod
    def kl_div(mu0, sig0, mu1, sig1):
        """
        Calcul de la divergence entre deux distributions normales paramétrisées par mu0, sig0, mu1, sig1
        """
        return -0.5 * torch.sum(1 - 2 * sig1.log() + 2 * sig0.log()
                                - (mu1 - mu0).pow(2) / sig1.pow(2) - (sig0/sig1).pow(2))

    def loss(self, obs, action):
        """"
        Définition du calcul de la Loss pour le réseau Deep Kalman Filter
        obs         : tableau des observations x_t
        time_step   : 2 ème dimension de obs, longueur de la fenêtre de temps, représente le nombre d'itération sur lesquelles sera entrainé
        batch_size  : 1ère dimension de obs, échantillon des données globales
        action      : tableau des actions intégrer dans le postérieur
        """

        time_step = obs.size(1)
        batch_size = obs.size(0)

        kl = torch.Tensor([0]).to(self.config.device)
        reconstruction = torch.Tensor([0]).to(self.config.device)
        
        # exponentiel du sigma précédemment défini
        decodeur_sig = self.decodeur_sigma.exp()
        # Ajustement de la dimension de l'espace latent en fonction des paramètres indiqués afin d'itérer sur les colonnes du batch
        z_q_t_1 = self.z_q_0.expand((batch_size-1, self.config.z_dim))
                
        for t in range(time_step):
            #Première prédiction faîte sur le premier espace latent
            decodeur_mu_0 = self.decodeur(self.z_q_0).reshape(1,1)
            # On traite l'espace latent avec le réseau de transition
            trans_mu, trans_sig = self.transition(z_q_t_1)
            # On ajuste notre distribution en prenant en compte les actions
            post_mu, post_sig = self.posterieur(trans_mu, trans_sig, action[:, t])

            z_q_t = self.reparametrization(post_mu, post_sig)
            decodeur_mu = self.decodeur(z_q_t)
            decodeur_mu = torch.cat((decodeur_mu_0,decodeur_mu))
            #print(decodeur_mu.shape,decodeur_mu[0],obs[:, t].shape)

            kl += self.kl_div(post_mu, post_sig, trans_mu, trans_sig)
            reconstruction += ((decodeur_mu - obs[:, t]).pow(2).sum(dim=0) / 2 / decodeur_sig
                               + self.decodeur_sigma * batch_size / 2).sum()

            z_q_t_1 = z_q_t

        return reconstruction, kl

## Entraînement

Définition des paramètres d'entraînement et entraînement sur les données.

In [ ]:
dim_cachee = 50

config_model = AttrDict({
                   "z_dim":2,
                   "decodeur_dim_cachee":dim_cachee,
                   "decodeur_dim_sortie":1,
                   "decodeur_constante":-5,
                   "transition_dim_cachee":dim_cachee,
                   "posterieur_dim_cachee":dim_cachee,
                   "actions_dim":U.shape[1],
                   "device": device,
                   "iter_num" : 500,
                   "beta":0.1
                   })

In [ ]:
dkf = DeepKalmanFilter(config_model)
optimizer = optim.Adam(dkf.parameters(), lr=0.001)
dkf.train()
dkf = dkf.to(device)

In [ ]:
train_loss, valid_losses = [],[]
with tqdm.tqdm(range(config_model.iter_num)) as pbar:
    for _ in pbar:
        
        optimizer.zero_grad()
        reconstruction, kl = dkf.loss(X_train_torch,U_train_torch)
        loss = reconstruction + config_model.beta * kl
        train_loss.append(loss)
        loss.backward()
        optimizer.step()
        with torch.no_grad():
            valid_reconstruction, valid_kl = dkf.loss(X_valid_torch,U_valid_torch)
        valid_loss = valid_reconstruction + config_model.beta * valid_kl
        valid_losses.append(valid_loss)
        pbar.set_postfix_str("[train kl %d, reconstruction %d, valid_kl %d, valid_reconst %d]"
                             % (kl, reconstruction, valid_kl, valid_reconstruction))

## Graphique sur le modèle

On cherche à tracer qqs graphiques sur les comportements du modèle et sur ses performances de prédictions sur la série temporelle

In [ ]:
plot_graphs = AttrDict({
                   "loss":True,
                   "test":True,
                   "aleatoire":False,
                   "entrainement":False,
                   "save_figure" : False
                   })


start = (N_ANNEE_DATASET-config_data.n_annee_validation-config_data.n_annee_entrainement)*N_UNITES_PAR_AN
end = config_data.n_annee_entrainement*N_UNITES_PAR_AN

In [ ]:
import os
filepath = f"./Graphs/epochs_{config_model.iter_num}_mois_{config_data.n_mois}_z_dim_{config_model.z_dim}_dimension_sous_réseaux_{dim_cachee}_beta_{config_model.beta}"
os.mkdir(filepath)

In [ ]:

def prediction_modele(U, model):
    z = []
    with torch.no_grad():
        z.append(model.decodeur(model.z_q_0).item())
        z_q_t_1 = model.z_q_0.expand((1, model.config.z_dim))        
        for u in U :
            prior_mu, prior_sig = model.transition(z_q_t_1)
            z_q_t, _ = model.posterieur(prior_mu, prior_sig, u)
            z_q_t_1 = z_q_t            
            z.append(model.decodeur(z_q_t_1).item())       
    return z

def mse(Y, YH):
    return '{:.3e}'.format(np.square(Y - YH).mean())

def projection_sur_test (title,Z_alpha,Y_alpha):
    fig = plt.figure()
    plt.plot(Z_alpha, label = 'Prediction')
    plt.plot(Y_alpha,label = 'Ground Truth')
    plt.legend()
    plt.title(title)
    plt.text(len(Y_alpha)+10,0.35,f'epochs :{config_model.iter_num} ,\nmois :{config_data.n_mois},\nz_dim : {config_model.z_dim},\ndim_cc : {dim_cachee},\nBeta :{config_model.beta},\nMSE : {mse(Z_alpha,Y_alpha)}')
    plt.show()
    if plot_graphs.save_figure :
        fig.savefig(filepath+'/'+title+'.jpg',bbox_inches='tight')
    
def prediction_plus_plot(title,U,X):
    Z = torch.Tensor(prediction_modele(U, dkf))
    projection_sur_test(title,Z,X)

In [ ]:
if plot_graphs.loss :
    
    valid_losses_cpu = [i.cpu() for i in valid_losses]
    train_loss_cpu = [i.cpu().detach().numpy() for i in train_loss]

    fig = plt.figure()
    plt.plot(train_loss_cpu,label='train loss')
    plt.plot(valid_losses_cpu,label='valid loss')

    #print(dkf.decodeur_sigma)

    plt.legend()
    plt.title('Combinaison des loss')
    plt.text(config_model.iter_num+30,train_loss_cpu[-1]/2, f'epochs :{config_model.iter_num} ,\nmois :{config_data.n_mois},\nz_dim : {config_model.z_dim},\ndim_cc : {dim_cachee}')
    plt.show()

    if plot_graphs.save:
        fig.savefig(filepath+'/Combinaison des loss.jpg',bbox_inches='tight')

In [ ]:
 if plot_graphs.test :

   U_alpha = torch.Tensor(Big_Data[['Inflow','Rainfall','Melt of Ice']].iloc[-start:-1].to_numpy()).to(device).reshape(start-1,1,3)
   X_alpha = Big_Data['Inflow'].iloc[-start:].to_numpy()

   title = 'Prédictions du modèle sur le jeu de test'
   prediction_plus_plot (title,U_alpha,X_alpha)

In [ ]:
if plot_graphs.aleatoire :
    
    U_beta = torch.rand((start,1,3)).to(config_model.device)
    X_beta = U_beta.cpu().reshape(start,3)[:,0]

    title = 'Prédictions du modèle sur un jeu aléatoire'
    prediction_plus_plot (title,U_beta[:-1],X_beta)

In [ ]:
if plot_graphs.entrainement :
    
    U_gamma = torch.Tensor(Big_Data[['Inflow','Rainfall','Melt of Ice']].iloc[end-start:end-1].to_numpy()).to(device).reshape(start-1,1,3)
    X_gamma = Big_Data['Inflow'].iloc[end-start:end].to_numpy()

    title = "Prédiction du model sur les 2 dernières années du jeu d'entraînement"
    prediction_plus_plot (title,U_gamma,X_gamma)